In [19]:
from astropy.io import fits, ascii
import numpy as np
import matplotlib.pyplot as plt
from utils.common_imports import np, time, plt, fits, Table, new_colors

In [ ]:
'''
catId
  A small integer specifying the source of the objId.  Currently the following catIds are defined.

    catId |  description
   -------|--------------------------------------------------
        0 | Simulated catalog
        1 | Gaia Data Release 1
        2 | Gaia Data Release 2
        3 | Gaia Early Data Release 3
        4 | Gaia Data Release 3
        5 | HSC-SSP Public Data Release 1 (Wide)
        6 | HSC-SSP Public Data Release 1 (Deep+UltraDeep)
        7 | HSC-SSP Public Data Release 2 (Wide)
        8 | HSC-SSP Public Data Release 2 (Deep+UltraDeep)
        9 | HSC-SSP Public Data Release 3 (Wide)
       10 | HSC-SSP Public Data Release 3 (Deep+UltraDeep)
       11 | HSC-SSP Public Data Release 4 (Wide)
       12 | HSC-SSP Public Data Release 4 (Deep+UltraDeep)
     1001 | Sky positions from S21A HSC-SSP (Wide)
     1002 | Sky positions from PS1
     1003 | Sky positions from Gaia
     1004 | Sky positions for regions without PS1 data
    90001 | Messier 15 stars for the engineering observation
    90002 | NGC1904 for the engineering run in November 2022
    90003 | NGC2419 for the engineering run in November 2022
    90004 | NGC5272 for the engineering run in February 2023
    90005 | NGC5904 for the engineering run in February 2023
    90006 | NGC7078 for the engineering run in July 2023
    90007 | NGC7089 for the engineering run in July 2023
    90008 | NGC7099 for the engineering run in July 2023

Comment: The combination of catId  and obj_id must be unique within one design.

Target uploader requirement:
https://pfs-etc.naoj.hawaii.edu/uploader/doc/inputs.html

ob_code, obj_id, obj_type, ra, dec, pmra, pmdec, epoch, exptime, priority, resolution, reference_arm, g_ps1, g_ps1_error
'''

In [2]:
# TODO: make this function useful for one/multiple pointings, plot the cosmology, star, sky and ancillary targets using different colors
def plot_tgt_done(outfn_list, figname, figsize=(8, 8), plot_diffcolor=True):
    '''
    plot the targets that have been assigned to the fibers
    '''
    for i, fn_i in enumerate(outfn_list):
        tgt_id_done, tgt_ra_done, tgt_dec_done = np.loadtxt(fn_i, usecols=(0, 4, 5), unpack=True, dtype='str')
        tgt_ra_done, tgt_dec_done = tgt_ra_done.astype('float'), tgt_dec_done.astype('float')

        mask_cos = [tgt_id_done[i][1:4] == 'Cos' for i in range(len(tgt_id_done))]
        mask_star = [tgt_id_done[i][1:4] == 'Sta' for i in range(len(tgt_id_done))]
        mask_sky = [tgt_id_done[i][1:4] == 'Sky' for i in range(len(tgt_id_done))]
        mask_anc = [tgt_id_done[i][1:4] == 'Anc' for i in range(len(tgt_id_done))]

        if(plot_diffcolor):
            if(i==0): 
                plt.figure(figsize = figsize)
                plt.plot(tgt_ra_done[mask_cos], tgt_dec_done[mask_cos], 'k.', ms=0.5, alpha=0.5, label='cosmology')
                plt.plot(tgt_ra_done[mask_star], tgt_dec_done[mask_star], 'r*', ms=1.5, alpha=1., label='star')
                plt.plot(tgt_ra_done[mask_sky], tgt_dec_done[mask_sky], 'b^', ms=1.5, alpha=1., label='sky')
                plt.plot(tgt_ra_done[mask_anc], tgt_dec_done[mask_anc], 'gs', ms=1.5, alpha=1., label='ancillary')
            else:
                plt.plot(tgt_ra_done[mask_cos], tgt_dec_done[mask_cos], 'k.', ms=0.5, alpha=0.5)
                plt.plot(tgt_ra_done[mask_star], tgt_dec_done[mask_star], 'r*', ms=1.5, alpha=1.)
                plt.plot(tgt_ra_done[mask_sky], tgt_dec_done[mask_sky], 'b^', ms=1.5, alpha=1.)
                plt.plot(tgt_ra_done[mask_anc], tgt_dec_done[mask_anc], 'gs', ms=1.5, alpha=1.)
        else:
            if(i==0): 
                plt.figure(figsize = figsize)
                plt.plot(tgt_ra_done, tgt_dec_done, 'k.', ms=0.5, alpha=0.5, label='targets done')
            else:
                plt.plot(tgt_ra_done, tgt_dec_done, 'k.', ms=0.5, alpha=0.5)

    plt.legend(loc='upper right', fontsize=15, frameon=True)
    plt.xlabel('RA', fontsize=15)
    plt.ylabel('DEC', fontsize=15)
    plt.savefig(figname, dpi=300, bbox_inches='tight')
    plt.show()
    plt.close()

# get the target list from netflow output and original cosmology target list

In [ ]:
fdir_raw_Jan25 = '../data_raw/Jan25/'
prefix_org = 'cmx_01222025_DESI-ELG_'
outdir_Jan25 = '../output/Jan25/' 
fdir_proc_Jan25 = '../data_proc/Jan25/'
prefix = ''

# HST(UTC-10) Jan 22 20:00 for field a and b, same for both visits
otime_list = ['2025-01-23T06:00:00Z', '2025-01-23T06:00:00Z']

#for field, Li_list, otime in zip(['a', 'b'], [['L1', 'L3', 'L4'], ['L1', 'L2', 'L3']], otime_list):
for field, Li_list, otime in zip(['a', ], [['L1', 'L3', 'L4'], ], ['2025-01-23T06:00:00Z', ]):
    infn_list = [] # for each filed, including all Li and all visits
    # read original cosmology targets
    data_cos = ascii.read(fdir_raw_Jan25 + prefix_org + 'field-{}_v{}.csv'.format(field, 1))
    # read processed star targets
    data_star = ascii.read(fdir_proc_Jan25 + 'star_field_{}_targets.ecsv'.format(field))
    # read processed sky targets
    data_sky = ascii.read(fdir_proc_Jan25 + 'sky_field_{}_targets.ecsv'.format(field))

    # create/add obj_code to the cosmology, star and sky targets
    obj_code_cos = np.array(['_Cos_'+str(x) for x in data_cos['obj_id']]) # revise
    obj_code_star = np.array(['_Sta_'+str(x) for x in data_star['ID']]) # revise
    obj_code_sky = np.array(['_Sky_'+str(x) for x in data_sky['ID']]) # revise

    # read netflow output
    for Li in Li_list[:1]:
        for visit in [1, 2][:1]:
            infn = outdir_Jan25 + prefix + 'field_%s_%s_visit%d.txt'%(field, Li, visit)
            infn_list.append(infn)
        
            tgt_id_done = np.loadtxt(infn, usecols=(0, ), unpack=True, dtype='str')
            # match the tgt_id_done cosmology with the original cosmology targets
            obj_id_cos = np.array(['_Cos_'+str(x) for x in data_cos['obj_id']])
            mask_cos = np.in1d(obj_id_cos, tgt_id_done) 
            
            # match the tgt_id_done star with the processed star targets
            mask_star = np.in1d(data_star['ID'], tgt_id_done) 

            # match the tgt_id_done sky with the processed sky targets
            mask_sky = np.in1d(data_sky['ID'], tgt_id_done)
            
            print('field %s, %s, visit %d, cosmology %d, star %d, sky %d'%(field, Li, visit, np.sum(mask_cos), np.sum(mask_star), np.sum(mask_sky)))
            # create obj_code for the targets
            
            # write all the matched targets to the final file
            


    figname = outdir_Jan25+prefix+'fiberassign_'+field+'_done.png'
    #plot_tgt_done(infn_list, figname, figsize=(8, 8), plot_diffcolor=True)

field a, Li L1, visit 1, cosmology 1665, star 200, sky 400


# reformat the data: necessary columns + other columns

In [31]:
print(data_cos.colnames)
print(data_cos.dtype)
print(data_star.colnames)
print(data_sky.colnames)

n_cos, n_star, n_sky = np.sum(mask_cos), np.sum(mask_star), np.sum(mask_sky)
print('field %s, %s, visit %d, cosmology--%d, star--%d, sky--%d'%(field, Li, visit, n_cos, n_star, n_sky))

['ob_code', 'obj_id', 'ra', 'dec', 'exptime', 'priority', 'resolution', 'reference_arm', 'g_hsc', 'g_hsc_err', 'r2_hsc', 'r2_hsc_err', 'i2_hsc', 'i2_hsc_err', 'z_hsc', 'z_hsc_err', 'y_hsc', 'y_hsc_err', 'matched']
[('ob_code', '<U26'), ('obj_id', '<i8'), ('ra', '<f8'), ('dec', '<f8'), ('exptime', '<f8'), ('priority', '<i8'), ('resolution', '<U1'), ('reference_arm', '<U1'), ('g_hsc', '<f8'), ('g_hsc_err', '<f8'), ('r2_hsc', '<f8'), ('r2_hsc_err', '<f8'), ('i2_hsc', '<f8'), ('i2_hsc_err', '<f8'), ('z_hsc', '<f8'), ('z_hsc_err', '<f8'), ('y_hsc', '<f8'), ('y_hsc_err', '<f8'), ('matched', '<U5')]
['ID', 'R.A.', 'Dec.', 'Exposure Time', 'stage', 'epoch', 'parallax', 'parallax_error', 'pmra', 'pmra_error', 'pmdec', 'pmdec_error', 'gPS1', 'rPS1', 'iPS1', 'zPS1', 'yPS1', 'gFluxJy', 'rFluxJy', 'iFluxJy', 'zFluxJy', 'yFluxJy', 'gFluxJy_err', 'rFluxJy_err', 'iFluxJy_err', 'zFluxJy_err', 'yFluxJy_err', 'flags_dist', 'flags_ebv', 'probfstar', 'catId']
['ID', 'R.A.', 'Dec.', 'Exposure Time', 'stage'

In [ ]:
# add catId to the cosmology targets with the value of 11 - HSC-SSP Public Data Release 4 (Wide), dytpes
data_cos.add_column(np.ones(len(data_cos), dtype=np.int32)*11, name='catId')

# rename 'ID', 'R.A.', 'Dec.' to 'obj_id', 'ra', 'dec' for the processed star and sky targets
for old_name, new_name in zip(['ID', 'R.A.', 'Dec.', 'Exposure Time'], ['obj_id', 'ra', 'dec', 'exptime']):
    data_star.rename_column(old_name, new_name)
    data_sky.rename_column(old_name, new_name)


# necessary columns for the final output
necessary_columns = ['ob_code', 'obj_id', 'catID', 'ra', 'dec', 'exptime', 'priority', 'resolution', 'reference_arm']
necessary_col_dtype = [str, np.int64, int, float, float, float, int, str, str]


# create the data table for the final output
data_table = Table()
for colname, coltype in zip(necessary_columns[:5], necessary_col_dtype[:5]):
    data_table[colname] = np.zeros(n_cos, dtype=coltype)
    data_table[colname][:n_cos] = data_cos[colname][mask_cos] 
    data_table[colname][n_cos:n_cos+n_star] = data_star[colname][mask_star]
    data_table[colname][n_cos+n_star:n_cos+n_star+n_sky] = data_sky[colname][mask_sky]

# create columns that share the same value for cosmology, star and sky targets
for colname, coltype, colval in zip(necessary_columns[5:], necessary_col_dtype[5:], [1, 'L', 'r']):
    # put data_cos[colname][mask_cos] to data_table[colname][:n_cos] with coltype
    data_table[colname] = np.zeros(n_cos, dtype=coltype) + colval

# create columns that have different colnames for cosmology, star and sky targets


# write the final output to a file
outfn = outdir_Jan25 + prefix + 'field_%s_%s_visit%d_final_targets.ecsv'%(field, Li, visit)
data_table.write(outfn, format='ascii.ecsv', overwrite=True)
print('write to %s'%outfn)



    

